# Dedalus script for 2D tracer adv-dif on a slope Analysis
## Ryan Holmes

In [ ]:
# Load modules
import numpy as np
import h5py
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
from scipy.special import erf
from scipy.interpolate import spline
from scipy import signal

import obsfit1d
import bbin

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('figure', facecolor='white')
plt.rc('font', size =14)
plt.rc('axes', facecolor='white')
plt.rc('savefig', facecolor='white')
plt.rc('xtick', direction='in')
plt.rc('ytick', direction='in')
plt.rc('xtick', top=True)
plt.rc('ytick', right=True)
panlabs = ['(a)','(b)','(c)','(d)','(e)','(f)','(g)','(h)','(i)','(j)','(k)','(l)','(m)','(n)','(o)','(p)','(q)']

## Analyse runs

In [ ]:
basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns24-08-18/';
savedir = '/home/z3500785/Cloud/Research/Data_Analysis/Bottom_Mixing_McDougall/Slope_Tracer_dedalus/Analysis_Figures/prodruns24-08-18/'
# # display file contents
# with h5py.File(basedir + 'z0_2_AH_000_slope_100/snapshots.h5', mode='r') as file:
#     print(list(file['scales'].keys()))
#     print(file['tasks/tr'])

### Plot Input fields

In [ ]:
runs = [basedir + 'z0_0p5000_AH_000_ADV_2_Kinf_m%s_slope_400/' % x for x in [3,5]] + [basedir + 'z0_0p5000_AH_000_ADV_2_Kinf_m5_slope_100/']# + [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [1]]
labs = [r'constant $\kappa$',r'exponential $\kappa$',r'slope=1/100']
params = np.load(runs[0] + 'runparams.npz');locals().update(params)
colors = ['k--','ko-','ko-'];

# Setup figure:
fig = plt.figure(figsize=(7, 5), facecolor='w')
axisV = plt.subplot(1,2,1)
axisV.set_xlabel(r'$V$ (cms$^{-1}$)')
axisV.set_ylabel(r'z (m)')
axisV.set_ylim([0.,250.])
axisV.plot([0., 0.],[0., 250.],':k',linewidth=1)
axisB = plt.subplot(1,2,2)
axisB.set_xlabel(r'$\frac{\partial B}{\partial z}$ ($10^{-6}$s$^{-2}$)')
axisB.set_ylim([0.,250.])
axisB.set_xlim([0.,1.1])
#axisB.set_title('Buoyancy Frequency')
# axisK = plt.subplot(1,3,3)
# axisK.set_xlabel(r'$\kappa$ (cm$^2$s$^{-1}$)')
# axisK.set_ylim([0.,Lz/5.])
# axisK.axes.get_yaxis().set_ticklabels([])
# axisK.grid()
# axisK.set_title('Isotropic Diffusivity')

for ii in range(0,len(runs)):
    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)
    with h5py.File(runs[ii] + 'ifields.h5', mode='r') as file:
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
        V = file['tasks/V'][0,0,:];K = file['tasks/K'][0,0,:];
        Bz = file['tasks/Bz'][0,0,:];
        h1 = axisV.plot(V*100.0,z.T,colors[ii], linewidth=1.,markerfacecolor='none');
        h2 = axisB.plot(Bz*1.e6,z.T,colors[ii], linewidth=1.,label=labs[ii],markerfacecolor='none');
        if ii == 2:
            h1[0].set_color('gray')
            h2[0].set_color('gray')
        # axisK.plot((K)/1.0e-4,z.T,colors[ii], linewidth=2.);
#axisV.legend(title='Slope')

# xlims = axisV.get_xlim();ylims = axisV.get_ylim();
# axisV.text(xlims[0]-0.3*(xlims[1]-xlims[0]),ylims[0]+0.95*(ylims[1]-ylims[0]),panlabs[0])
# xlims = axisB.get_xlim();ylims = axisB.get_ylim();
# axisB.text(xlims[0]-0.15*(xlims[1]-xlims[0]),ylims[0]+0.95*(ylims[1]-ylims[0]),panlabs[1])
# xlims = axisK.get_xlim();ylims = axisK.get_ylim();
# axisK.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.93*(ylims[1]-ylims[0]),panlabs[2])

axisB.legend(loc=(-0.93,0.72),frameon=False)
# plt.tight_layout()
plt.savefig(savedir + 'Input_VKBz.pdf',bbox_inches='tight')

### Plot Tracer Distribution

In [ ]:
# Combination Plotting:
names = ['z0_0p5000_AH_000_ADV_%s_Kinf_m5_slope_400' % x for x in [0,1,2]]
titles = ['no advection','BBL advection only','full advection']

plts = [1,4,2,5,3,6]
fig = plt.figure(figsize=(13, 7))
for fi in range(0,len(names)):
    name = names[fi]
    outdir = basedir + name + '/'
    params = np.load(outdir + 'runparams.npz');locals().update(params)
    snaps = outdir + 'snapshots.h5';
    tts = [50,100]
    for ii in range(len(tts)):
        axisTR = plt.subplot(2,3,plts[fi*2+ii])
        with h5py.File(snaps, mode='r') as file:
            y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
            dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
            dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);
            ym, zm = np.meshgrid(y,z)
            zt = np.cos(theta)*zm + np.sin(theta)*ym;
            yt = -np.sin(theta)*zm + np.cos(theta)*ym # Rotated Coordinates
            B = N2*np.sin(theta)*ym + N2*np.cos(theta)*(zm + np.exp(-q0*zm)*np.cos(q0*zm)/q0)
            t = file['scales/sim_time'][:]/lday;
            trT = file['tasks']['trT'][:,0,0]
            tr = file['tasks']['tr'][tts[ii],:,:]
            tr[np.where(tr<0.)] = 0.
            tr0 = file['tasks']['tr'][0,:,:]
            zm1 = file['tasks']['zm1i'][tts[ii],0,0];
            muz = zm1/trT
            ym1 = file['tasks']['ym1i'][tts[ii],0,0];
            muy = ym1/trT
            mx = 0.1
            p = axisTR.contourf(yt/1.e3, zt, tr.T, cmap='magma_r', levels=np.arange(0.,mx+mx/20,mx/20),extend='max')
            plt.plot((-np.sin(theta)*cz + np.cos(theta)*cy)/1e3,np.cos(theta)*cz+np.sin(theta)*cy,'ow',mew=2,ms=8)
            plt.plot((-np.sin(theta)*muz + np.cos(theta)*muy)/1e3,np.cos(theta)*muz+np.sin(theta)*muy,'xw',mew=2,ms=8)
            
        # Add buoyancy contours
        plt.contour(yt/1.0e3, zt, B, 30, colors='k')
        axisTR.plot(y/1.0e3, slope*y,'k-', linewidth=1.)
        if fi == 0:
            axisTR.set_ylabel(r'$z_t$ (m)');
        if ii == 1:
            axisTR.set_xlabel(r'$y_t$ (km)');
        axisTR.set_facecolor('k')
        axisTR.set_ylim([1650.,3500.])
        axisTR.set_xlim([500.,1200.])
        xlims = axisTR.get_xlim()
        ylims = axisTR.get_ylim()
        txt = axisTR.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.92*(ylims[1]-ylims[0]),panlabs[plts[fi*2+ii]-1] + ' ' + titles[fi])
        axisTR.text(xlims[0]+0.73*(xlims[1]-xlims[0]),ylims[0]+0.05*(ylims[1]-ylims[0]),'day %d' % t[tts[ii]],color='white')

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.25, 0.015, 0.5])
cbar = fig.colorbar(p, cax=cbar_ax)
cbar.set_ticks(np.arange(0.,0.12,0.02))
plt.savefig(savedir + 'TrConcMgma_ADV.png',dpi=300,bbox_inches='tight')    


### Buoyancy distribution and moments

In [ ]:
rc('font', size =15)
fig = plt.figure(figsize=(10, 7.5), facecolor='w')
ab = [];
nex = 3;
ab.append(plt.subplot2grid((nex,2),(0,0), colspan=1, rowspan=nex))
for i in np.arange(1,nex+1):
    ab.append(plt.subplot2grid((nex,2),(i-1,1)));
ab[0].set_ylabel(r'$b\,\, (10^{-4} ms^{-2})$');ab[0].set_xlabel('Concentration (tr / (ms$^{-2}$))');
ab[1].set_ylabel(r'$\mu_b\,\, (10^{-4} ms^{-2})$');#ab[1].text(set_title('Centre of mass $\mu_b$');
ab[2].set_ylabel(r'$\kappa\,\, (cm^2s^{-1})$');#ab[3].set_title('Effective diffusivity from second moment');
ab[3].set_ylabel(r'$\overline{\kappa}^t\,\, (cm^2s^{-1})$');#ab[3].set_title('Effective diffusivity from second moment');
ab[nex].set_xlabel('Days')
for i in np.arange(1,nex+1):
    ab[i].grid();
panlabs = [r'a',r'b',r'c',r'd',r'e',r'f',r'g',r'h',r'i',r'j',r'k',r'l']

# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [2,1,0]]
# leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']
# runs = [basedir + 'z0_' + x + '_AH_000_Kinf_m5_ADV_2_slope_400/' for x in ['1p0000','0p5000','0p1250']]
# leglabel = ['$z_0=d=500$m','$z_0=d/2=250$m','$z_0=d/8=62.5$m']
# runs = [basedir + 'z0_0p5000_AH_%03d/' % x for x in [0,10,100]]
# leglabel = ['No Isopycnal Diffusion','$A_H=10$m$^2$s$^{-1}$','$A_H=100$m$^2$s$^{-1}$']
# runs = [basedir + 'ADV_%s_AH_000_slope_200/' % x for x in [2,1,0]]
# leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']
# runs = [basedir + 'ADV_2_AH_%03d_slope_200_maxy_10000000_miny_600/' % x for x in [10,30,100]]
# leglabel = ['$A_H=10$m$^2$s$^{-1}$','$A_H=30$m$^2$s$^{-1}$','$A_H=100$m$^2$s$^{-1}$']
# runs = [basedir + 'z0_0p5000_AH_%03d_ADV_0_Kinf_m3/' % x for x in [100]]
# runs = runs + [basedir + 'z0_0p5000_AH_100_ADV_%01d/' % x for x in [0,1]];
# runs = runs + [basedir + 'z0_0p5000_AH_100/']
# leglabel = ['Constant Isotropic Diffusivity','Bottom-Intensified Isotropic Diffusivity']#,'Kinf = -5']
# leglabel = leglabel + ['Bot-Intens. + BBL Advection','Bot-Intens. + SML + BBL Advection']
# runs = [basedir + 'z0_0p5000_AH_100/'] + [basedir + 'z0_0p5000_AH_100_ADV_%01d/' % x for x in [1,0]]
# leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']

# runs = [basedir + 'z0_0p5000_AH_100_ADV_0_Kinf_m3/',basedir + 'z0_0p5000_AH_100_ADV_0/',basedir + 'z0_0p5000_AH_010_ADV_0/']
# leglabel = [r'Const. $\kappa$, $A_H=100$m$^2$s$^{-1}$',r'B-int. $\kappa$, $A_H=100$m$^2$s$^{-1}$',r'B-int. $\kappa$, $A_H=10$m$^2$s$^{-1}$']

# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_%03d/' % x for x in [100,200,400]]
# leglabel = ['1/100','1/200','1/400']
# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_400' + x + '/' for x in ['','_4dt','_4dt_2dz']]# + [basedir + 'z0_0p5000_AH_050_Kinf_m5_ADV_2_slope_400' + x + '/' for x in ['','_4dt']]
# leglabel = ['$\Delta t=8$ days, 128 z modes','$\Delta t=2$ days, 128 z modes','$\Delta t=2$ days, 256 z modes']#'$\Delta t=8$ days, $A_H=50$m$^2$s$^{-1}$','$\Delta t=2$ days, $A_H=50$m$^2$s$^{-1}$']
# runs = [basedir + 'ADV_2_AH_100_slope_%03d/' % x for x in [100, 200]]
# leglabel = ['Slope = 1/100','Slope = 1/200']

# runs = [basedir + 'ADV_0_AH_000_slope_200_Kinf_m3_maxy_10000000_miny_600/'];
# leglabel = ['']

#runs = [basedir + 'z0_0p5000_AH_100_ADV_2_Kinf_m5_slope_400_AHdd/',basedir + 'z0_0p5000_AH_100_ADV_2_Kinf_m5_slope_400/',
runs = [basedir + 'z0_0p5000_AH_100_ADV_0_Kinf_m5_slope_400_AHvar_1/',basedir + 'z0_0p5000_AH_100_ADV_0_Kinf_m5_slope_400_AHvar_0/']
#leglabel = ['$A_H$ BBL reduced','$A_H$ Constant','$A_H$ BBL reduced no ADV','$A_H$ Constant no ADV']
leglabel = ['$A_H$ BBL reduced no ADV','$A_H$ Constant no ADV']

# runs = [basedir + 'z0_0p5000_AH_100_ADV_2_Kinf_m5_slope_400/'];
# runs = [basedir + 'z0_0p5000_AH_000_ADV_0_Kinf_m3_slope_400/'];
# leglabel = ['Base'];

colors = ['g-','r-','b-','k-','c-','g--','r--','b--'];

# tifin = 200 # Final time index
# tinds = [x//16 for x in [0,1000,2000,3000]]
# tifin = 101 # Final time index
# tinds = [x//16 for x in [0,200,400,1600]]
tifin = 49 # Final time index
tinds = [x for x in [0,10,20,30,40]]
# tifin = 101 # Final time index
# tinds = [x//16 for x in [200,1600]]

dop3f = False#True
doKhadj = False#True
doKhw = False

if doKhw:
    fig2, ax = plt.subplots()
    fig2.set_facecolor('white')

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)
    
    if dop3f:
        par3fits = np.load(runs[ii] + 'par3fitGOODINI.npz');
        sucinds = par3fits['suc'] == 1
        sucinds[176] = False

    with h5py.File(runs[ii] + 'ifields.h5', mode='r') as file:
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        # Plot Profiles:
        t = file['scales/sim_time'][:]/lday;
        trI = file['tasks']['tr'][0,:,:]
        b, trIp = bbin.trbinI(y,z,trI,N2,theta,Lz,Ly,q0)
        zF = b/N2;dzF = zF[1]-zF[0];
        imax = np.argmax(trIp)
        zF = zF - zF[imax]

        xlims = [0.,t[tifin-1]]
        for i in tinds:
            tr = file['tasks']['tr'][i,:,:]
            b, trp = bbin.trbinI(y,z,tr,N2,theta,Lz,Ly,q0)
            if i == tinds[0]:
                ab[0].plot(signal.savgol_filter(trp,5,2),b/1.e-4,colors[ii],label=leglabel[ii],linewidth=2)
            else:
                ab[0].plot(signal.savgol_filter(trp,5,2),b/1.e-4,colors[ii],linewidth=2)
                if (dop3f):
                    trfit = obsfit1d.trMOD([par3fits['K'][i],par3fits['Kh'][i],
                                            par3fits['w'][i]],zF,trp,trIp, int(t[i]*lday))
                    #if ii == 1:
                    # ab[0].plot(trfit,b/1.e-4,colors[ii].replace('-',':'),linewidth=2)


    with h5py.File(runs[ii] + 'moments.h5', mode='r') as file:
        t = file['scales/sim_time'][:]/lday;
        trT = file['tasks']['trT'][:,0,0]

        # Plot Time-Series:
        com = file['tasks']['bm1i'][:,0,0]/trT
        Vcom = file['tasks']['VtrBy'][:,0,0]/trT
        Kcomy = -file['tasks']['KtryBy'][:,0,0]/trT
        Kcomz = -file['tasks']['KtrzBz'][:,0,0]/trT
        Kzcom = -file['tasks']['KztrBZ'][:,0,0]/trT
        Kcomzbp = -file['tasks']['KtrzBzp'][:,0,0]/trT
        # Kbblycom = -file['tasks']['KbblTtryBy'][:,0,0]/trT
        # Kbblzcom = -file['tasks']['KbblTtrzBz'][:,0,0]/trT
        # Vbblcom = file['tasks']['VbbltrBy'][:,0,0]/trT
        # VbblTcom = file['tasks']['VbblTtrBy'][:,0,0]/trT
        # AHcom = -file['tasks']['AHbm1'][:,0,0]/trT

        # ab[1].plot(t,Vcom,'-k')
        # ab[1].plot(t,Kzcom,'--r')
        ab[1].plot(t,com/1.e-4,colors[ii],linewidth=2)
        # ab[1].plot(t,com[0]/1.e-4 + np.cumsum(Vcom)*dt/1.e-4,'-k',linewidth=3)
        # # ab[1].plot(t,com[0]/1.e-4 + np.cumsum(Vbblcom)*dt/1.e-4,colors[ii].replace('-',':'),linewidth=3)
        # # ab[1].plot(t,com[0]/1.e-4 + np.cumsum(VbblTcom)*dt/1.e-4,colors[ii].replace('-','-.'),linewidth=3)
        # ab[1].plot(t,com[0]/1.e-4 + np.cumsum(Kcomy)*dt/1.e-4,colors[ii].replace('-',':'),linewidth=1)
        # ab[1].plot(t,com[0]/1.e-4 + np.cumsum(Kcomz)*dt/1.e-4,'-r',linewidth=1)
        # ab[1].plot(t,com[0]/1.e-4 + np.cumsum(Kzcom)*dt/1.e-4,'-b')
        # ab[1].plot(t,com[0]/1.e-4 + np.cumsum(Kcomzbp)*dt/1.e-4,'-m')
        # # ab[1].plot(t,com[0]/1.e-4 + np.cumsum(AHcom)*dt/1.e-4,colors[ii].replace('-',':'),linewidth=3)

        bm2 = file['tasks']['bm2i'][:,0,0]
        Ktr = file['tasks']['Ktr'][:,0,0];
        Katrz = file['tasks']['Katrz'][:,0,0];
        atrzT = file['tasks']['atrzT'][:,0,0]

        sigma2 = bm2/trT-com**2.
        sigma2nomu = bm2/trT
        Keff = 0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/N2/N2/1.e-4;
        Ktrw = Ktr/trT/1e-4
        Ktrgw = Katrz/atrzT/1e-4
        ab[2].plot((t[1:]+t[:-1])/2,Keff,colors[ii],linewidth=2)
        ab[2].plot(t,Ktrw,colors[ii].replace('-','--'),linewidth=2)
        # ab[2].plot(t,Ktrgw,colors[ii].replace('-',':'),linewidth=3)
        KeffTA = [Keff[0]] + [np.sum(np.array(Keff[:i+1])/(i+1)) for i in range(0,len(Keff))]
        KtrwTA = [np.sum(np.array(Ktrw[:i+1])/(i+1)) for i in range(0,len(Ktrw))]
        KtrgwTA = [np.sum(np.array(Ktrgw[:i+1])/(i+1)) for i in range(0,len(Ktrgw))]
        ab[3].plot(t,KeffTA,colors[ii],linewidth=2);
        ab[3].plot(t,KtrwTA,colors[ii].replace('-','--'),linewidth=2);
        if (dop3f):
            p3t = par3fits['t']
            p3K = np.array([x/1e-4 for x in par3fits['K']])
            if (doKhadj):
                p3Kh = np.array([x/1e-7 for x in par3fits['Kh']])
                p3w = np.array([-x/1e-7 for x in par3fits['w']])+2*p3Kh # I had the sign of D1 wrong!!
                ab[3].plot(p3t[sucinds],p3K[sucinds]+p3Kh[sucinds]*(p3w[sucinds]+p3Kh[sucinds])*p3t[sucinds]*1.e-5,colors[ii].replace('-','-.').replace(':','-.'),linewidth=2);
            else:
                ab[3].plot(p3t[sucinds],p3K[sucinds],colors[ii].replace('-','-.').replace(':','-.'),linewidth=2);

ab[0].legend()
# # ADV case:
# ab[0].set_ylim([5.,25.]);ab[0].set_xlim([0.,1.e10])
# ab[1].set_ylim([10.,18.]);ab[2].set_ylim([0.,7.5]);
# ab[3].set_ylim([0.,7.5])
# z0 case:
# ab[0].set_ylim([6.,18.]);ab[0].set_xlim([0.,1.8e10])
# ab[1].set_ylim([8.,15.]);ab[2].set_ylim([0.,10.]);
# ab[3].set_ylim([0.,10.])
# ab[1].axes.yaxis.set_ticks([8.,10.,12.,14.])
# AH100 ADV case:
ab[0].set_ylim([15.,28.]);ab[0].set_xlim([0.,1.3e10])
#ab[1].set_ylim([20.,23.]);ab[2].set_ylim([0.,7.]);
ab[3].set_ylim([0.,7.])
# # AH100 NOADV case:
# ab[0].set_ylim([15.,28.]);ab[0].set_xlim([0.,1.3e10])
# ab[1].set_ylim([20.,22.5]);ab[2].set_ylim([0.,11.]);
# ab[3].set_ylim([0.,11.])
#
# # Layer case AH=100:
# ab[0].set_ylim([33.,53.]);ab[0].set_xlim([0.,0.6e11])
# ab[1].set_ylim([42.5,45.5]);ab[2].set_ylim([2.,7.]);
# ab[1].axes.yaxis.set_ticks([43.,44.,45.])
# ab[3].set_ylim([0.,8.])
# Layer case Kinf=-3
# ab[0].set_ylim([25.,60.]);ab[0].set_xlim([0.,0.6e11])
# ab[1].set_ylim([44,50.]);ab[2].set_ylim([0.,15.]);

for i in np.arange(1,nex+1):
    ab[i].set_xlim(xlims);
    xlims = ab[i].get_xlim();ylims = ab[i].get_ylim();
    ab[i].text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.88*(ylims[1]-ylims[0]),panlabs[i],fontsize=15,fontweight='bold')
for i in np.arange(1,nex):
    ab[i].axes.xaxis.set_ticklabels([])

xlims = ab[0].get_xlim();ylims = ab[0].get_ylim();
ab[0].text(xlims[0]+0.08*(xlims[1]-xlims[0]),ylims[0]+0.96*(ylims[1]-ylims[0]),panlabs[0],fontsize=15,fontweight='bold')

if doKhw:
    ax.set_xlabel('Days')
    ax.set_ylabel(r'$(cm day^{-1})$');#ab[3].set_title('Effective diffusivity from second moment');
    ax.grid()
    ax.set_xlim([0., 2000.])
    ax.set_title(r'1D fit $\overline{w}$ (dashed) and $\overline{\kappa}_h$ (solid)')

# ab[0].set_title('Tracer Distribution Days 0 and 1600')

# ab[4].set_xlim([0., 1000.])
# ab[4].set_ylim([-50., 25.])
# Plt.tight_layout()
# plt.savefig(savedir + 'bSpaceAnalysis_AH100_ADV.pdf');


In [ ]:
tst = np.array([1.,1.,1.,1.])
np.cumsum(tst)         

In [ ]:
# Testing COM and Var budgets:
rc('font', size =15)
fig, ax = plt.subplots(1,1,figsize=(10,8))
fig.set_facecolor('white')
ax.set_ylabel(r'$b\,\, (10^{-4} ms^{-2})$');
ax.set_xlabel('Time (days)')

run = basedir + 'z0_0p5000_AH_100_ADV_0_Kinf_m5_slope_400_AHvar_1/';
params = np.load(run + 'runparams.npz');locals().update(params)
    
with h5py.File(run + 'ifields.h5', mode='r') as file:
    y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];

with h5py.File(run + 'moments.h5', mode='r') as file:

    t = file['scales/sim_time'][:]/lday;
    trT = file['tasks']['trT'][:,0,0]

    # Plot COM:
    com = file['tasks']['bm1i'][:,0,0]/trT
    Vcom = file['tasks']['VtrBy'][:,0,0]/trT
    Kcomz = -file['tasks']['KtrzBz'][:,0,0]/trT
    Kzcom = file['tasks']['KztrBZ'][:,0,0]/trT
    Kcomzbp = -file['tasks']['KtrzBzp'][:,0,0]/trT
    Kbblcomz = -file['tasks']['KbblTtrzBz'][:,0,0]/trT
    Vbblcom = file['tasks']['VbbltrBy'][:,0,0]/trT
    VbblTcom = file['tasks']['VbblTtrBy'][:,0,0]/trT
    AHcom = -file['tasks']['AHbm1'][:,0,0]/trT
    Kcomy = -file['tasks']['KtryBy'][:,0,0]/trT

    ax.plot(t,com/1.e-4,'-k',linewidth=4,label=r'$\langle b\Theta\rangle/\langle\Theta\rangle$')
    # Shift these by a time step!!::
    ax.plot(t,com[0]/1.e-4 + np.cumsum(Kcomz+Vcom+Kcomy)*dt/1.e-4,'--m',linewidth=4,label=r'$\int \langle \Theta V b_y\rangle - \langle\kappa\nabla\Theta\cdot\nabla b \rangle dt$')
    ax.plot(t,com[0]/1.e-4 + np.cumsum(Vcom)*dt/1.e-4,'-b',linewidth=4,label=r'$\int \langle \Theta V b_y\rangle dt$')
    ax.plot(t,com[0]/1.e-4 + np.cumsum(Kcomz)*dt/1.e-4,'--r',linewidth=4,label=r'$-\int \langle \kappa\nabla\cdot\Theta\nabla b\rangle dt$')
    ax.plot(t,com[0]/1.e-4 + np.cumsum(Kzcom)*dt/1.e-4,'--r',linewidth=3,label=r'$\int N^2\cos\theta\langle \Theta\kappa_z\rangle dt$')
    ax.plot(t,com[0]/1.e-4 + np.cumsum(Kcomzbp)*dt/1.e-4,':r',linewidth=3,label=r"$-\int \kappa\Theta_z b_z' dt$")
    ax.plot(t,com[0]/1.e-4 + np.cumsum(Kcomz-Kzcom-Kcomzbp)*dt/1.e-4,'-r',linewidth=3,label=r'$\int N^2\kappa_0\cos\theta\int_{-\infty}^\infty\Theta(y,0,t)dydt$')
    ax.plot(t,com[0]/1.e-4 + np.cumsum(Vbblcom)*dt/1.e-4,'--b',linewidth=3,label=r'$\int \langle \Theta V_{BBL} b_y\rangle dt$')

    ax.plot(t,com[0]/1.e-4 + np.cumsum(Kbblcomz)*dt/1.e-4,'-.r',linewidth=1,label='Kbblcomz')
    ax.plot(t,com[0]/1.e-4 + np.cumsum(VbblTcom)*dt/1.e-4,':g',linewidth=1,label='VbblTcom')
    ax.plot(t,com[0]/1.e-4 + np.cumsum(AHcom)*dt/1.e-4,'-m',linewidth=3,label='AHcom')

    # ax.plot((t[1:]+t[:-1])/2.,(com[1:]-com[:-1])/dt,'-k',linewidth=4,label='COM')
    # ax.plot(t,Kcomz+Vcom+Kcomy,'--m',linewidth=3,label='Kcomz+Vcom')
    # ax.plot(t,Vcom,'-g',linewidth=2,label='Vcom')
    # ax.plot(t,Kcomz,'-r',linewidth=2,label='Kcomz')
    # ax.plot(t,Kcomy,'-b',linewidth=2,label='Kcomy')
    # ax.plot(t,Kzcom,'--r',linewidth=1,label='Kzcom')
    # ax.plot(t,Kcomzbp,':r',linewidth=1,label='Kcomzbp')
    # ax.plot(t,Kbblcomz,'-.r',linewidth=1,label='Kbblcomz')
    # ax.plot(t,Vbblcom,'--g',linewidth=1,label='Vbblcom')
    # ax.plot(t,VbblTcom,':g',linewidth=1,label='VbblTcom')
    # ax.plot(t,AHcom,'-m',linewidth=3,label='AHcom')

    ax.legend()

    ax.set_ylim([20.,24.])
    ax.set_xlim([0, 800.])
    ax.set_ylim([21.,21.5])
    ax.set_xlim([0, 100.])

#     # Plot Second moment:
#     var = file['tasks']['bm2i'][:,0,0]/trT
#     Vvar = 2.*file['tasks']['VtrBBy'][:,0,0]/trT
#     Kvary = -2.*file['tasks']['KtryBBy'][:,0,0]/trT
#     Kvarz = -2.*file['tasks']['KtrzBBz'][:,0,0]/trT
#     Vbblvar = 2.*file['tasks']['VbbltrBBy'][:,0,0]/trT
#     Kpvarz = -2.*file['tasks']['KtrzBBzp'][:,0,0]/trT
#     Kzvar = -2.*file['tasks']['KztrBBZ'][:,0,0]/trT
#     Ktr = file['tasks']['Ktr'][:,0,0];
# #    Ktr = file['tasks']['Ktr'][:,0,0];
    
#     ax.plot(t,var/1.e-4,'-k',linewidth=4,label=r'$\langle b^2\Theta\rangle/\langle \Theta\rangle$')
#     ax.plot(t,var[0]/1.e-4 + np.cumsum(Vvar+Kvary+Kvarz)*dt/1.e-4,'--m',linewidth=4,label='Sum')
#     ax.plot(t,var[0]/1.e-4 + np.cumsum(Vvar)*dt/1.e-4,'-b',linewidth=4,label='Total Adv')
#     ax.plot(t,var[0]/1.e-4 + np.cumsum(Kvarz)*dt/1.e-4,'--r',linewidth=4,label='Z-diff')
#     ax.plot(t,var[0]/1.e-4 + np.cumsum(Vbblvar)*dt/1.e-4,'--b',linewidth=3,label='BBL Adv')
#     ax.plot(t,var[0]/1.e-4 + np.cumsum(Kpvarz)*dt/1.e-4,'--r',linewidth=2,label="Z-diff b'")
#     ax.plot(t,var[0]/1.e-4 + np.cumsum(Kzvar)*dt/1.e-4,':r',linewidth=2,label="Z-diff Kz effect")
#     ax.set_xlim([0, 800.])
#     ax.set_ylim([0.0425, 0.05])
#     ax.legend()
    
# plt.savefig(savedir + 'z0_0p5000_AH_000_ADV_2_Kinf_m5_slope_400_M2balance.pdf')


In [ ]:
# B distribution and moments Large time series
fig = plt.figure(figsize=(8, 12), facecolor='w')
ab = []
for i in range(0,5):
    ab.append(plt.subplot2grid((5,1),(i,0)))
ab[0].set_ylabel(r'$b\,\, (10^{-4} $ms$^{-2})$');
ab[1].set_ylabel(r'$b\,\, (10^{-4} $ms$^{-2})$');
ab[2].set_ylabel(r'$b\,\, (10^{-4} $ms$^{-2})$');
ab[0].grid();ab[1].grid();ab[2].grid();ab[3].grid();ab[4].grid()
# ab[3].set_ylabel(r'Time-Average $\kappa (cm^2s^{-1})$');
# ab[4].set_ylabel(r'Time-Average $\overline{w}$, $\overline{\kappa}_h$ (cm day$^{-1}$)');
ab[3].set_ylabel(r'cm$^2$s$^{-1}$');
ab[4].set_ylabel(r'cm day$^{-1}$');
ab[4].set_xlabel('Days')

runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [2,1,0]]
leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']
colors = ['g-','r-','b-','k-','y-','m-'];
panlabs = ['a ' + leglabel[0],'b ' + leglabel[1],'c ' + leglabel[2],'d','e','f','g','h','i','j','k','l']
#runs = [runs[1]]

# pfr = 10 # Profile frequency
# tifin = 100 # Final time index
# concsc = 200./1.e10 # Concentration to time conversion scale
pfr = 20 # Profile frequency
tifin = 199 # Final time index
concsc = 400./1.e10 # Concentration to time conversion scale

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)
    par3fits = np.load(runs[ii] + 'par3fitGOODINI.npz');
    sucinds = par3fits['suc'] == 1
    sucinds[176] = 0

    with h5py.File(runs[ii] + 'ifields.h5', mode='r') as file:
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        trI = file['tasks']['tr'][0,:,:]
        b, trIp = bbin.trbinI(y,z,trI,N2,theta,Lz,Ly,q0)
        zF = b/N2;dzF = zF[1]-zF[0];
        imax = np.argmax(trIp)
        zF = zF - zF[imax]

        xlims = [0.,t[tifin]]
        for i in range(0,tifin,pfr):#len(t),pfr):
            tr = file['tasks']['tr'][i,:,:]
            b, trp = bbin.trbinI(y,z,tr,N2,theta,Lz,Ly,q0)
            ab[ii].plot(signal.savgol_filter(trp,5,2)*concsc+t[i],b/1.e-4,colors[ii])
            if (i > 1):
                trfit = obsfit1d.trMOD([par3fits['K'][i],par3fits['Kh'][i],
                                        par3fits['w'][i]],zF,trp,trIp, int(t[i]*lday))
                ab[ii].plot(signal.savgol_filter(trfit,5,2)*concsc+t[i],b/1.e-4,colors[ii].replace('-','--'))
            
    with h5py.File(runs[ii] + 'moments.h5', mode='r') as file:

        trT = file['tasks']['trT'][:,0,0]
        zm1 = file['tasks']['zm1i'][:,0,0];
        muz = zm1/trT

        bm1 = file['tasks']['bm1i'][:,0,0];bm2 = file['tasks']['bm2i'][:,0,0]
        Ktr = file['tasks']['Ktr'][:,0,0];Katrz = file['tasks']['Katrz'][:,0,0];atrzT = file['tasks']['atrzT'][:,0,0]
        mu = bm1/trT
        sigma2 = bm2/trT-(bm1/trT)**2.
        sigma2nomu = bm2/trT
        Keff = 0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/N2/N2/1.e-4;
        Ktrw = Ktr/trT/1e-4
        Ktrgw = Katrz/atrzT/1e-4
        ab[ii].plot(t,mu/1.e-4,colors[ii].replace('-',':'),linewidth=3)
        # ab[3].plot(t,np.sqrt(sigma2)/1.e-4,colors[ii])
        # ab[3].plot((t[1:]+t[:-1])/2,Keff,colors[ii])
        # ab[3].plot(t,Ktrw,colors[ii].replace('-','--'))
#        ab[4].plot(t,Katrgw,colors[ii].replace('-',':'),linewidth=3)
        KeffTA = [Keff[0]] + [np.sum(np.array(Keff[:i+1])/(i+1)) for i in range(0,len(Keff))]
        KtrwTA = [np.sum(np.array(Ktrw[:i+1])/(i+1)) for i in range(0,len(Ktrw))]
        KtrgwTA = [np.sum(np.array(Ktrgw[:i+1])/(i+1)) for i in range(0,len(Ktrgw))]
#        ab[3].plot(t,KeffTA,colors[ii],linewidth=2);
#        ab[3].plot(t,KtrwTA,colors[ii].replace('-','--'));
        p3t = par3fits['t']
        p3K = np.array([x/1e-4 for x in par3fits['K']])
        p3Kp = (np.array(par3fits['K'])+2*np.array(par3fits['Kerr']))/1e-4
        p3Km = (np.array(par3fits['K'])-2*np.array(par3fits['Kerr']))/1e-4
        if (ii == 1):
            ab[3].plot(p3t[sucinds],p3K[sucinds],'-k',linewidth=3,label=r'$\overline{\kappa}_0$');
        ab[3].plot(p3t[sucinds],p3K[sucinds],colors[ii],linewidth=3);
        ab[3].plot(p3t[sucinds],p3Kp[sucinds],colors[ii],linewidth=1);
        ab[3].plot(p3t[sucinds],p3Km[sucinds],colors[ii],linewidth=1);
        p3Kh = np.array([x/1e-7 for x in par3fits['Kh']])
        p3w = np.array([-x/1e-7 for x in par3fits['w']])+2*p3Kh # I had the sign of D1 wrong!!
        p3Khp = (-np.array(par3fits['Kh'])+2*np.array(par3fits['Kherr']))/1e-7
        p3Khm = (-np.array(par3fits['Kh'])-2*np.array(par3fits['Kherr']))/1e-7
        p3wp = (-np.array(par3fits['w'])+2*np.array(par3fits['werr']))/1e-7
        p3wm = (-np.array(par3fits['w'])-2*np.array(par3fits['werr']))/1e-7
        if (ii == 1):
            ab[3].plot(p3t[sucinds],p3K[sucinds]+p3Kh[sucinds]*(p3w[sucinds]+p3Kh[sucinds])*p3t[sucinds]*1.e-5,'--k',linewidth=3,label=r'$\overline{\kappa}_0+\overline{\kappa}_h\mu_h$')
            ab[4].plot(p3t[sucinds],p3w[sucinds],'-k',linewidth=3,label=r'$\overline{w}$');
            ab[4].plot(p3t[sucinds],p3Kh[sucinds],'--k',linewidth=3,label=r'$\overline{\kappa}_h$');
        ab[3].plot(p3t[sucinds],p3K[sucinds]+p3Kh[sucinds]*(p3w[sucinds]+p3Kh[sucinds])*p3t[sucinds]*1.e-5,colors[ii].replace('-','--'),linewidth=3)
        ab[4].plot(p3t[sucinds],p3Kh[sucinds],colors[ii].replace('-','--'),linewidth=3);
        # ab[4].plot(p3t[sucinds],p3Khp[sucinds],colors[ii],linewidth=1);
        # ab[4].plot(p3t[sucinds],p3Khm[sucinds],colors[ii],linewidth=1);
        ab[4].plot(p3t[sucinds],p3w[sucinds],colors[ii],linewidth=3);
        # ab[4].plot(p3t[sucinds],p3w[sucinds],colors[ii].replace('-','--'),linewidth=1);
        # ab[4].plot(p3t[sucinds],p3w[sucinds],colors[ii].replace('-','--'),linewidth=1);
        ab[4].set_ylim([-10.,15.])

ab[3].legend()
ab[4].legend(loc=(0.85,0.001))
for i in range(3):
    ab[i].set_ylim([7.5, 20.])
    
for i in range(5):
    ab[i].set_xlim(xlims)
    xlimsL = ab[i].get_xlim();ylimsL = ab[i].get_ylim();
    if i<=2:
        ab[i].text(xlimsL[0]+0.01*(xlimsL[1]-xlimsL[0]),ylimsL[0]+0.85*(ylimsL[1]-ylimsL[0]),panlabs[i],fontsize=15,fontweight='bold')
    else:
        ab[i].text(xlimsL[0]+0.01*(xlimsL[1]-xlimsL[0]),ylimsL[0]+0.05*(ylimsL[1]-ylimsL[0]),panlabs[i],fontsize=15,fontweight='bold')

for i in range(4):
    ab[i].axes.xaxis.set_ticklabels([])
# plt.tight_layout()
plt.savefig(savedir + 'bSpaceAnalysis_ExampleFits.pdf');

# Summary Figure of effective diffusivity

In [ ]:
rc('font', size =15)
fig = plt.figure(figsize=(8, 5), facecolor='w')
ax = plt.gca()
ax.set_xlabel(r'Along-isopycnal Diffusivity (m$^2$s$^{-1}$)');
ax.grid()

fig2 = plt.figure(figsize=(8, 5), facecolor='w')
ax2 = plt.gca()
ax2.set_xlabel(r'Along-isopycnal Diffusivity (m$^2$s$^{-1}$)');
ax2.grid()

fig3 = plt.figure(figsize=(8, 5), facecolor='w')
ax3 = plt.gca()
ax3.set_xlabel(r'Along-isopycnal Diffusivity (m$^2$s$^{-1}$)');
ax3.grid()

base = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/'

layer = True
if layer:
    # Layer-releases:
    runs = [base + 'prodruns_bIC30-5-19/ADV_%s_AH_000_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]]
    colors = ['g','r','b']
    types = ['o','o','o']
    labels = ['SML + BBL Adv','BBL Adv','No Adv']
    runs.extend([base + 'prodruns_bIC30-5-19/ADV_%s_AH_010_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o'] * 3)
    labels.extend([''] * 3)
    runs.extend([base + 'prodruns_bIC30-5-19/ADV_%s_AH_020_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o'] * 3)
    labels.extend([''] * 3)
    runs.extend([base + 'prodruns_bIC30-5-19/ADV_%s_AH_030_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o'] * 3)
    labels.extend([''] * 3)
    runs.extend([base + 'prodruns_bIC30-5-19/ADV_%s_AH_040_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o'] * 3)
    labels.extend([''] * 3)
    runs.extend([base + 'prodruns_bIC30-5-19/ADV_%s_AH_050_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o'] * 3)
    labels.extend([''] * 3)
    runs.extend([base + 'prodruns_bIC30-5-19/ADV_%s_AH_060_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o'] * 3)
    labels.extend([''] * 3)
    runs.extend([base + 'prodruns_bIC30-5-19/ADV_%s_AH_075_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o'] * 3)
    labels.extend([''] * 3)
    runs.extend([base + 'prodruns_bIC30-5-19/ADV_%s_AH_090_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o'] * 3)
    labels.extend([''] * 3)
    runs.extend([base + 'prodruns_bIC30-5-19/ADV_%s_AH_100_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o'] * 3)
    labels.extend([''] * 3)
    runs.extend([base + 'prodruns_bIC30-5-19/ADV_%s_AH_150_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o'] * 3)
    labels.extend([''] * 3)
else:
    # Point-releases:
    runs = [base + 'prodruns24-5-19/z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [2,1,0]]
    colors = ['g','r','b']
    types = ['o','o','o']
    labels = ['SML + BBL Adv','BBL Adv','No Adv']
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_%03d/' % x for x in [5,10,50,100]])
    colors.extend(['g'] * 4)
    types.extend(['o'] * 4)
    labels.extend([''] * 4)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_100_ADV_%01d/' % x for x in [1,0]])
    colors.extend(['r','b'])
    types.extend(['o','o'])
    labels.extend(['']*2)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_005_ADV_%01d/' % x for x in [1,0]])
    colors.extend(['r','b'])
    types.extend(['o','o'])
    labels.extend(['']*2)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_050_ADV_%01d/' % x for x in [1,0]])
    colors.extend(['r','b'])
    types.extend(['o','o'])
    labels.extend(['']*2)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_010_ADV_%01d/' % x for x in [1,0]])
    colors.extend(['r','b'])
    types.extend(['o','o'])
    labels.extend(['']*2)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_030_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_075_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_150_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_020_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_030_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_040_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_060_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_090_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_125_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_175_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    runs.extend([base + 'prodruns_wide30-5-19/z0_0p5000_AH_200_ADV_%01d/' % x for x in [2,1,0]])
    colors.extend(['g','r','b'])
    types.extend(['o','o','o'])
    labels.extend(['']*3)
    # types.extend(['d','d','d','d','d'])

tind = 50
dop3f = False#True

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)
    
    if dop3f:
        par3fits = np.load(runs[ii] + 'par3fitGOODINI.npz');
        sucinds = par3fits['suc'] == 1

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        trT = file['tasks']['trT'][:,0,0]
        bm1 = file['tasks']['bm1i'][:,0,0];bm2 = file['tasks']['bm2i'][:,0,0]
        # bm1 = file['tasks']['zm1i'][:,0,0];bm2 = file['tasks']['zm2i'][:,0,0]
        Ktr = file['tasks']['Ktr'][:,0,0];Katrz = file['tasks']['Katrz'][:,0,0];atrzT = file['tasks']['atrzT'][:,0,0]
        mu = bm1/trT
        sigma2 = bm2/trT-(bm1/trT)**2.
        sigma2nomu = bm2/trT
        Keff = 0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/N2/N2/1.e-4;
        # Keff = 0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/1.e-4;
        Ktrw = Ktr/trT/1e-4
        # Ktrw = Ktrw + AH*np.sin(theta)*np.sin(theta)/1.e-4
        Ktrgw = Katrz/atrzT/1e-4
        KeffTA = [Keff[0]] + [np.sum(np.array(Keff[:i+1])/(i+1)) for i in range(0,len(Keff))]
        KtrwTA = [np.sum(np.array(Ktrw[:i+1])/(i+1)) for i in range(0,len(Ktrw))]
        KtrgwTA = [np.sum(np.array(Ktrgw[:i+1])/(i+1)) for i in range(0,len(Ktrgw))]
        if (len(labels[ii])>0):
            ax.plot(AH,Keff[tind],types[ii],color=colors[ii],markersize=12,label=labels[ii])
        else:
            ax.plot(AH,Keff[tind],types[ii],color=colors[ii],markersize=12)
        if (ii == 0):
            ax.plot(AH,KtrwTA[tind],'x',color=colors[ii],markersize=12,linewidth=3,label='In-situ')
        else:
            ax.plot(AH,KtrwTA[tind],'x',color=colors[ii],markersize=12,linewidth=3)
        if (dop3f):
            if (ii == 0):
                ax.plot(AH,par3fits['K'][tind]/1.e-4,'>',color=colors[ii],markersize=8,linewidth=3,label='Least-squares 1D')
            else:
                ax.plot(AH,par3fits['K'][tind]/1.e-4,'>',color=colors[ii],markersize=8,linewidth=3)
        if (len(labels[ii])>0):
            ax2.plot(AH,(mu[tind]-mu[0])/1.e-4,types[ii],color=colors[ii],markersize=12,label=labels[ii])
        else:
            ax2.plot(AH,(mu[tind]-mu[0])/1.e-4,types[ii],color=colors[ii],markersize=12)
        if (len(labels[ii])>0):
            ax3.plot(AH,(bm2[tind] - 2*mu[0]*bm1[tind]+mu[0]*mu[0]*trT[tind]),types[ii],color=colors[ii],markersize=12,label=labels[ii])
        else:
            ax3.plot(AH,(bm2[tind] - 2*mu[0]*bm1[tind]+mu[0]*mu[0]*trT[tind]),types[ii],color=colors[ii],markersize=12)

# ax.legend(loc=(0.02,0.47))
ax.legend()
ax.set_ylabel(r'$\overline{\kappa}^t$ Day %04d $(cm^2s^{-1})$' % t[tind]);

ax2.legend()
ax2.set_ylabel(r'$\mu_b$ Day %04d $(10^{-4} $ms$^{-2})$' % t[tind]);

ax3.legend()
ax3.set_ylabel(r'$\langle (b-b_0)^2 \Theta\rangle$ Day %04d $(10^{-4} $ms$^{-2})$' % t[tind]);

svdir = '/home/z3500785/Cloud/Research/Data_Analysis/Bottom_Mixing_McDougall/Slope_Tracer_dedalus/Analysis_Figures/'
# plt.savefig(svdir + 'Summary_Plot_b2m_Day800.pdf');


###   Z distribution and moments

In [ ]:
fig = plt.figure(figsize=(8, 6), facecolor='w')
gs = gridspec.GridSpec(2, 2, width_ratios=[1., 1.5])
gs.update(wspace=0.3)
ab0 = plt.subplot(gs[:,0])
ab1 = plt.subplot(gs[0,1])
ab2 = plt.subplot(gs[1,1])
ab0.set_ylabel('z (m)');ab0.set_xlabel(r'$\langle\Theta\rangle^y$ (tracer m$^{-1}$)');
ab1.set_ylabel(r'$\mu_z\,\, (m)$');
ab2.set_ylabel(r'$cm^2s^{-1}$');
ab2.set_xlabel('Days');ab3.grid()

runs = [basedir + 'z0_0p5000_AH_000_ADV_2_Kinf_m%s_slope_400/' % x for x in [3,5]]
leglabel = [r'constant $\kappa$',r'exponential $\kappa$']
styles = ['-','-']
colors = ['k','gray']

tifin = 25 # Final time index
tinds = [x for x in [0,3,12]]
tlabs = []

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)

    with h5py.File(runs[ii] + 'ifields.h5', mode='r') as file:
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        ym0 = file['tasks']['ym0']

        xlims = [0.,t[tifin]]
        for i in tinds:
            if i == 0:
                ab0.plot(ym0[i,0,:],z,styles[ii],color=colors[ii],label=leglabel[ii])
            else:
                ab0.plot(ym0[i,0,:],z,styles[ii],color=colors[ii])
            if ii==0:
                tlabs.append('day %d' % t[i])

    with h5py.File(runs[ii] + 'moments.h5', mode='r') as file:
        t = file['scales/sim_time'][:]/lday;

        zm1 = file['tasks']['zm1i'][:,0,0];
        zm2 = file['tasks']['zm2i'][:,0,0]
        trT = file['tasks']['trT'][:,0,0]
        Ktr = file['tasks']['Ktr'][:,0,0]
        Katrz = file['tasks']['Katrz'][:,0,0]
        atrzT = file['tasks']['atrzT'][:,0,0]
        mu = zm1/trT
        sigma2 = zm2/trT-(zm1/trT)**2.
        Keff = 0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/1e-4
        ab1.plot(t,mu,styles[ii],color=colors[ii])
        if ii == 0:
            ab2.plot((t[1:]+t[:-1])/2,Keff,styles[ii],color=colors[ii],label=r'$\kappa_{eq}$')
            ab2.plot(t,Ktr/trT/1e-4,'--',color=colors[ii],label=r'$\langle\kappa\Theta\rangle/\langle\Theta\rangle$')
        else:
            ab2.plot((t[1:]+t[:-1])/2,Keff,styles[ii],color=colors[ii])
            ab2.plot(t,Ktr/trT/1e-4,'--',color=colors[ii])
        
        # # Analytic solution for constant kappa:
        # sigA = np.sqrt(sz**2 + 2*K0*t*lday)
        # muA = 2*sigA/np.sqrt(2*np.pi)*np.exp(-cz**2/2/sigA**2.) + cz*erf(cz/np.sqrt(2*sigA**2))
        # m2A = cz**2+sigA**2
        # sigAeff = m2A - muA**2
        # KeffA = K0*(1.-2./np.pi*np.exp(-cz**2/sigA**2) - 2.*cz/np.sqrt(2.*np.pi)/sigA*erf(cz/np.sqrt(2*sigA**2))*np.exp(-cz**2/2/sigA**2))
        
        # ab1.plot(t,muA,'--c')
        # ab2.plot(t,KeffA/1.e-4,'--c')

# Kinf cases:
ab0.set_xlim([0, 30000.])
ab0.set_ylim([0., 500.])
ab1.set_xlim(xlims)
ab1.set_ylim([230., 310.])
ab2.set_xlim(xlims)
ab2.set_ylim([0,11.])
ab0.legend(frameon=False,loc=(0.18,0.85))
ab2.legend(frameon=False)

xlims = ab0.get_xlim();ylims = ab0.get_ylim();
ab0.text(xlims[0]-0.29*(xlims[1]-xlims[0]),ylims[0]+0.97*(ylims[1]-ylims[0]),panlabs[0])
xlims = ab1.get_xlim();ylims = ab1.get_ylim();
ab1.text(xlims[0]-0.2*(xlims[1]-xlims[0]),ylims[0]+0.93*(ylims[1]-ylims[0]),panlabs[1])
xlims = ab2.get_xlim();ylims = ab2.get_ylim();
ab2.text(xlims[0]-0.2*(xlims[1]-xlims[0]),ylims[0]+0.93*(ylims[1]-ylims[0]),panlabs[2])

ab0.text(2.2e4,295.,tlabs[0])
ab0.text(1.6e4,240.,tlabs[1])
ab0.text(0.75e4,50.,tlabs[2])
plt.savefig(savedir + 'zSpaceAnalysis_Kinf.pdf',bbox_inches='tight')


# Analytic Solution Constant Mixing with Boundary

In [ ]:
# Constant Mixing Time Series:
t = np.linspace(0.,800,500)
K0 = 1.e-3
sz = 50.
cz = 250.

# Analytic solution for constant kappa:
sigA = np.sqrt(sz**2 + 2*K0*t*lday)
muA = 2*sigA/np.sqrt(2*np.pi)*np.exp(-cz**2/2/sigA**2) + cz*erf(cz/np.sqrt(2*sigA**2))
m2A = cz**2+sigA**2
sigAeff = m2A - muA**2
KeffA = K0*(1.-2./np.pi*np.exp(-cz**2/sigA**2) - 2.*cz/np.sqrt(2.*np.pi)/sigA*erf(cz/np.sqrt(2*sigA**2))*np.exp(-cz**2/2/sigA**2))

# # Time series:
# f = plt.figure(figsize=(10,5))
# f.set_facecolor('white')
# plt.plot(t,muA)

# Animation of two gaussians:
z = np.linspace(0.,2000.,500)
for t in np.linspace(0,1000,200):
    sigA = np.sqrt(sz**2 + 2*K0*t*lday)
    sol1 = sz/sigA*np.exp(-(z-cz)**2/2./sigA**2)
    sol2 = sz/sigA*np.exp(-(z+cz)**2/2./sigA**2)

    f = plt.figure(figsize=(5,5))
    f.set_facecolor('white')

    plt.plot(sol1,z,'-r',label='Upper Gaussian')
    plt.plot(sol2,z,'-b',label='Lower Gaussian')
    plt.plot(sol1+sol2,z,'-k',label='Total')
    plt.xlabel('Amplitude')
    plt.ylabel('Height (m)')
    plt.grid()
    plt.title('Time = %03d days' % t)
    plt.xlim(-0.001,0.5)
    plt.ylim(0.,1000.)
    plt.legend()
    plt.savefig('frame%03d.png' % t)

### Y Distribution and moments

In [ ]:
# Plot Y distribution and moments:
fig = plt.figure(figsize=(10, 5), facecolor='w')
ab0 = plt.subplot2grid((3,2),(0,0), colspan=1, rowspan=3)
ab1 = plt.subplot2grid((3,2),(0,1));
ab2 = plt.subplot2grid((3,2),(1,1));
ab3 = plt.subplot2grid((3,2),(2,1));
ab0.set_xlabel('y (km)');ab0.set_ylabel('Tracer Concentration (tr / m)');
ab0.set_title('Tracer Distribution');ab0.set_xlim([0.,Ly/1.e3]);
ab1.set_ylabel('y (km)');ab1.set_title('Centre of mass $\mu_y$');
ab1.grid()
ab2.set_ylabel('y (km)');ab2.set_title('Standard deviation $\sigma_y$');
ab2.grid()
ab3.set_ylabel(r'$\kappa^y_{eff} (m^2s^{-1})$');ab3.set_title('Effective diffusivity from second moment');
ab3.set_xlabel('Days')
ab3.grid()
panlabs = ['a','b','c','d','e','f','g','h','i','j','k','l']

# runs = [basedir + 'z0_' + x + '_AH_000_Kinf_m5_ADV_2_slope_400/' for x in ['1p0000','0p5000','0p1250']]
# leglabel = ['$z_0=d=500$m','$z_0=d/2=250$m','$z_0=d/8=62.5$m']
# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m%s_ADV_2_slope_400/' % x for x in [3,4,5]] + [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [0,1]]
# leglabel = [x.replace(basedir + 'z0_0p5000_AH_000_','').replace('_slope_400/','').replace('_',' ').replace('Kinf m','$\kappa_\infty = 10^{-').replace(' ADV 2','}m^2s^{-1}$ ').replace(' ADV 1','}m^2s^{-1}$ BBL V only').replace('ADV 0','}m^2s^{-1}$ no ADV') for x in runs]
# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_%03d/' % x for x in [100,200,400]]
# leglabel = ['1/100','1/200','1/400']
# runs = [basedir + 'z0_0p5000_AH_%03d/' % x for x in [0,10,100]]
# leglabel = ['No Isopycnal Diffusion','$A_H=10$m$^2$s$^{-1}$','$A_H=100$m$^2$s$^{-1}$']
# runs = [basedir + 'z0_0p5000_AH_%03d_Kinf_m5_ADV_2_slope_400/' % x for x in [0,1,5,10,50,100]]
# leglabel = ['%sm$^2$s$^{-1}$' % x for x in [0,1,5,10,50,100]]
# colors = ['k-','r-','b-','g-','y-','m-'];
# runs = [basedir + 'ADV_%s_AH_000_slope_200/' % x for x in [2,1,0]]
# leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']
runs = [basedir + 'ADV_%01d_AH_000_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]]
runs = runs + [basedir + 'ADV_%01d_AH_100_slope_200_maxy_10000000_miny_600/' % x for x in [2,1,0]]
leglabel = ['SML and BBL Advection $A_H=0$m$^2$s$^{-1}$','BBL Advection Only $A_H=0$m$^2$s$^{-1}$','No Advection $A_H=0$m$^2$s$^{-1}$']
leglabel = leglabel + ['SML and BBL Advection $A_H=100$m$^2$s$^{-1}$','BBL Advection Only $A_H=100$m$^2$s$^{-1}$','No Advection $A_H=100$m$^2$s$^{-1}$']
colors = ['g-','r-','b-','g--','r--','b--'];

pfr = 16
tifin = 199

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        zm0 = file['tasks']['zm0']

        xlims = [0.,t[tifin]]
        for i in range(0,tifin,pfr):#len(t),pfr):
            if i == 0:
                ab0.plot(y/1.e3,zm0[i,:,0],colors[ii],label=leglabel[ii])
            else:
                ab0.plot(y/1.e3,zm0[i,:,0],colors[ii])
#        ab0.legend(title='Horizontal Diffusivity $A_H$')
#        ab0.legend(title='Initial Tracer Height $z_0$')
        ab0.legend()
    
        ym1 = file['tasks']['ym1i'][:,0,0];
        ym2 = file['tasks']['ym2i'][:,0,0]
        trT = file['tasks']['trT'][:,0,0]
        mu = ym1/trT
        sigma2 = ym2/trT-(ym1/trT)**2.
        ab1.plot(t,mu/1.e3,colors[ii])
        ab2.plot(t,np.sqrt(sigma2)/1.e3,colors[ii])
        ab3.plot((t[1:]+t[:-1])/2,0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday,colors[ii])
        # print(np.max(0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday))


ab3.set_xlim(xlims)
ab1.set_xlim(xlims)
ab2.set_xlim(xlims)

#ab0.set_xlim([300.,1100.])
#ab0.set_ylim([0.,50.])
xlims = ab0.get_xlim();ylims = ab0.get_ylim();
ab0.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.95*(ylims[1]-ylims[0]),panlabs[0],fontsize=15,fontweight='bold')
xlims = ab1.get_xlim();ylims = ab1.get_ylim();
ab1.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.83*(ylims[1]-ylims[0]),panlabs[1],fontsize=15,fontweight='bold')
xlims = ab2.get_xlim();ylims = ab2.get_ylim();
ab2.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.83*(ylims[1]-ylims[0]),panlabs[2],fontsize=15,fontweight='bold')
xlims = ab3.get_xlim();ylims = ab3.get_ylim();
ab3.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.83*(ylims[1]-ylims[0]),panlabs[3],fontsize=15,fontweight='bold')
plt.tight_layout()
# plt.savefig(savedir + 'ySpaceAnalysis_AH.pdf');


# Diapycnal Fluxes

In [ ]:
fig = plt.figure(figsize=(10, 5), facecolor='w')
ab0 = plt.subplot(1,2,1);
ab1 = plt.subplot(1,2,2);
ab0.set_ylabel(r'$b\,\, (10^{-4} ms^{-2})$');ab0.set_xlabel('Tracer Concentration (tr / (ms$^{-2}$))');
ab0.set_title('Tracer Distribution Day 320')
ab0.set_ylim([5.,20.]);
ab1.set_ylabel(r'$b\,\, (10^{-4} ms^{-2})$');ab0.set_xlabel('Diapycnal Tracer Flux (tr / (s$^{-2}$))');
ab1.set_title('Diapycnal Tracer Flux Day 320')
ab1.set_ylim([5.,20.]);
ab1.grid()            
panlabs = ['a','b','c','d','e','f','g','h','i','j','k','l']

# runs = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_%s_slope_400/' % x for x in [2,1,0]]
# leglabel = ['SML and BBL Advection','BBL Advection Only','No Advection']
runs = [basedir + 'z0_' + x + '_AH_000_Kinf_m5_ADV_2_slope_400/' for x in ['1p0000','0p5000','0p1250']]
leglabel = ['$z_0=d=500$m','$z_0=d/2=250$m','$z_0=d/8=62.5$m']
colors = ['g-','r-','b-','k-','y-','m-'];
# runs = [runs[1]]

pfr = 20
tifin = 21

for ii in range(0,len(runs)):

    params = np.load(runs[ii] + 'runparams.npz');locals().update(params)
    b = np.linspace(0,N2*np.cos(theta)*Lz + N2*np.sin(theta)*Ly,64);db = b[1]-b[0]
    ba = (b[:-1]+b[1:])/2

    with h5py.File(runs[ii] + 'ifields.h5', mode='r') as file:
        B = file['tasks']['B'][0,:,:]
        B = (B[1:,:] + B[:-1,:])/2
        B = (B[:,1:] + B[:,:-1])/2
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
    dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
    dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);

    with h5py.File(runs[ii] + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        dt = (t[1]-t[0])*lday;
        trT = file['tasks']['trT'][:,0,0]

        mub = np.zeros(len(t))
        sigma2b = np.zeros(len(t))
        xlims = [0.,t[tifin]]
        for i in [20]:#range(0,tifin,pfr):#len(t),pfr):
            tr = file['tasks']['tr'][i,:,:]
            tr = (tr[1:,:] + tr[:-1,:])/2
            tr = (tr[:,1:] + tr[:,:-1])/2
            tr = tr*dA
            trCm = np.array([np.sum(tr[np.logical_and(B>=bi,B<(bi+db))])/db for bi in b[:-1]])
            tr = file['tasks']['tr'][i+1,:,:]
            tr = (tr[1:,:] + tr[:-1,:])/2
            tr = (tr[:,1:] + tr[:,:-1])/2
            tr = tr*dA
            trCp = np.array([np.sum(tr[np.logical_and(B>=bi,B<(bi+db))])/db for bi in b[:-1]])

            # bpl = np.linspace(b.min()/1.e-4,b.max()/1.e-4,300)
            # trC_smooth = spline((b[:-1]+b[1:])/2/1.e-4,(trCp+trCm)/2.,bpl);
            if i == 20:
                ab0.plot((trCp+trCm)/2.,(b[:-1]+b[1:])/2/1.e-4,colors[ii],label=leglabel[ii])
                # ab0.plot(trC_smooth,bpl,colors[ii],label=leglabel[ii])
            else:
                ab0.plot((trCp+trCm)/2.,(b[:-1]+b[1:])/2/1.e-4,colors[ii])
                # ab0.plot(trC_smooth,bpl,colors[ii])
            dPdt = (trCp-trCm)/dt
            trF = -np.cumsum(dPdt*db)
            ab1.plot(trF,(b[:-1]+b[1:])/2/1.e-4,colors[ii])
#            ab1.plot(dPdt,(b[:-1]+b[1:])/2/1.e-4,colors[ii])
        ab0.legend()
        ab0.set_xlim([0.,1.0e10])
        ab1.set_xlim([-0.02,0.04])

xlims = ab0.get_xlim();ylims = ab0.get_ylim();
ab0.text(xlims[0]+0.07*(xlims[1]-xlims[0]),ylims[0]+0.95*(ylims[1]-ylims[0]),panlabs[0],fontsize=15,fontweight='bold')
xlims = ab1.get_xlim();ylims = ab1.get_ylim();
ab1.text(xlims[0]+0.02*(xlims[1]-xlims[0]),ylims[0]+0.83*(ylims[1]-ylims[0]),panlabs[1],fontsize=15,fontweight='bold')

plt.tight_layout()
plt.savefig(savedir + 'bSpaceFluxes_z0.pdf');


In [ ]:
# Analytic solution plotting:

In [ ]:
# parameters:
d = 500.;kappa = 1.e-3;AH = 100.
mu0z = d/2;mu0y = 0.
sig0z = 10.;sig0y = 100.
slope = 1/400.;

# Derived parameters:
theta = np.arctan(slope);
K11 = kappa + AH*np.cos(theta)**2.
K12 = - AH*np.sin(theta)*np.cos(theta)
K22 = kappa + AH*np.sin(theta)**2.

# Time axis:
t = np.linspace(0.,1000.,100.)*1.e5

# Moment solution time series:
sig2 = sig0z**2 + 2.*K22*t;
sig = np.sqrt(sig2);
trT = 2.*np.pi*sig0z*sig0y;
# muz = 2*sig/np.sqrt(2.*pi)*np.exp(-mu0z**2./2./sig2)+mu0z*erf(mu0z/np.sqrt(2.*sig2)); #INCORRECT NORMALIZATION
# sigz2 = mu0z**2.+sig2-muz**2. #INCORRECT NORMALIZATION

muy = (mu0y + trT*K12/K22*(2*sig*(np.exp(-mu0z**2/2/sig2)-np.exp(-mu0z**2./2./sig0z**2.))+np.sqrt(2*np.pi)*mu0z*(erf(mu0z/np.sqrt(2*sig2))-erf(mu0z/np.sqrt(2.*sig0z**2.)))))/trT

plt.plot(t,muy)




### Analytic plot of B derivatives to check BBL approximations for isopycnal mixing

In [ ]:
theta = np.arctan(1./100.);N2 = 1.e-6;K0 = 1.e-3;Pr0=1.
q0 = (N2*np.sin(theta)*np.sin(theta)/4.0/Pr0/K0/K0)**(1.0/4.0)

z = np.linspace(0.,2./q0,200.)
f  = np.exp(-q0*z)*(np.cos(q0*z)+np.sin(q0*z))
fz = -2.*q0*np.exp(-q0*z)*np.sin(q0*z)
V = -K0*fz/np.tan(theta);
V = V/np.max(V)

Bz = 1-f
Bz2d = (1-f)**2./(np.tan(theta)**2.+(1-f)**2.)
ByBzd = (1-f)/(np.tan(theta)+(1./np.tan(theta))*(1-f)**2.)
By2d = 1/(1+(1./np.tan(theta))**2.*(1-f)**2.)
ByBzdd = fz*((1./np.tan(theta))*(1-f)**2.-np.tan(theta))/(np.tan(theta)+(1./np.tan(theta))*(1-f)**2.)**2.
By2dd = 2*(1-f)*fz/(1+(1./np.tan(theta))**2.*(1-f)**2.)**2.

fig = plt.figure(figsize=(5,7))
fig.set_facecolor('white')
plt.plot(Bz,z,'-k',linewidth=3,label=r'$B_z/N^2\cos{\theta}$')
plt.plot(V,z,'--k',linewidth=3,label=r'$V/max(V)$')
plt.plot(Bz2d,z,'-r',linewidth=2,label=r'$B_z^2/|\nabla B|^2$')
plt.plot(ByBzd,z,'-b',linewidth=2,label=r'$B_z B_y/|\nabla B|^2$')
plt.plot(By2d,z,'-m',linewidth=2,label=r'$B_y^2/|\nabla B|^2$')
plt.plot(ByBzdd,z,'-c',linewidth=2,label=r'$\frac{\partial}{\partial z}(B_z B_y/|\nabla B|^2)$')
plt.plot(By2dd,z,'-g',linewidth=2,label=r'$\frac{\partial}{\partial z}(B_y^2/|\nabla B|^2)$')
plt.plot([0.,1.],[1./q0,1./q0],'-k')
plt.legend()
ax = plt.gca()
ax.set_ylabel('Height $z$ (m)')
ax.set_ylim([0.,np.max(z)])
plt.text(0.03,1./q0+3,r'$\tan{\theta} = 1/%3.0f$' % (1./np.tan(theta)),fontsize='x-large')
plt.text(0.03,1./q0+1,r'$q_0^{-1} = %2.1f$m' % (1./q0),fontsize='x-large')
plt.savefig('IsopycnalMixing_Bgradients_Sl100.png')


# Ununsed/Old plotting code

In [ ]:
# Testing new buoyancy binning code:
import imp
imp.reload(bbin)
run = basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_400/'

params = np.load(run + 'runparams.npz');locals().update(params)

with h5py.File(run + 'ifields.h5', mode='r') as file:
    B = file['tasks']['B'][0,:,:]
    y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];

with h5py.File(run + 'snapshots.h5', mode='r') as file:

    t = file['scales/sim_time'][:]/lday;
    trT = file['tasks']['trT'][:,0,0]
    
    i = 1
    tr = file['tasks']['tr'][i,:,:]

b, trp = bbin.trbinI(y,z,tr,N2,theta,Lz,Ly,q0)

f = plt.figure(figsize=(10,5),facecolor='white')
ax0 = plt.subplot2grid((1,3),(0,0))
ax0.plot(trp,b,'.-k')
ax0.set_ylim([0., N2*np.cos(theta)*Lz*2])

ym, zm = np.meshgrid(y,z)
zt = np.cos(theta)*zm + np.sin(theta)*ym;
yt = -np.sin(theta)*zm + np.cos(theta)*ym # Rotated Coordinates
B = N2*np.sin(theta)*ym + N2*np.cos(theta)*(zm + np.exp(-q0*zm)*np.cos(q0*zm)/q0)

axisTR = plt.subplot2grid((1,3),(0,1), colspan=2, rowspan=1)
p = axisTR.contourf(yt/1.e3, zt, tr.T, cmap='magma_r', levels=np.arange(0.,np.max(tr),np.max(tr)/26),extend='both')#np.max(tr)/);#GnBu
axisTR.set_ylim([0., Lz*2])

# Add buoyancy contours
plt.contour(yt/1.0e3, zt, B, 15, colors='k')
axisTR.plot(y/1.0e3, slope*y,'k-', linewidth=4.)
plt.colorbar(p, ax = axisTR)
#plt.title(run + 'Day %d' % t[i])